In [1]:
import pickle

In [2]:
x = []
y = []

In [3]:
import numpy as np
def flatten(d):    
    res = []  # Result list
    if isinstance(d, dict):
        for key, val in d.items():
            if key == 'controller' or key == 'action_counter':
                pass
            res.extend(flatten(val))
    elif isinstance(d, list):
        for i in d:
            if isinstance(i, dict):
                res.extend(flatten(i))
            else:
                res.extend(i)
    elif isinstance(d, np.ndarray):
        d = d.tolist()     
        for i in d:
            if isinstance(i, dict):
                res.extend(flatten(i))
            else:
                res.append(i)
    else:
        raise TypeError("Undefined type for flatten: %s"%type(d))

    return res

def getpair(experience):
    f = flatten(experience['state'])
    #f.extend(experience['prev_action'].tolist())
    return f, experience['action'][len(experience['action']) - 1]

In [4]:
with open('/home/victor/ML/Smash/phillip-master/agents/FalconFalconBF/experience/320404a83c3644f6aab384a5668ed19a_102','rb') as f:
    experience = pickle.load(f)
experience
#s, a = getpair(experience)
#a

{'action': array([25, 25, 25, 25, 25, 25, 25, 25, 25, 25,  0, 24, 25, 25, 25, 26,  3,
         3,  9, 24, 24,  4,  4,  4,  2,  2, 11, 11, 22,  5]),
 'global_step': 10588863,
 'initial': [],
 'prev_action': array([25, 25, 25, 25, 25, 25, 25, 25, 25, 25, 25,  0, 24, 25, 25, 25, 26,
         3,  3,  9, 24, 24,  4,  4,  4,  2,  2, 11, 11, 22]),
 'prob': array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  0.05480213,
         0.02635193,  0.00741132,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  0.0471715 ,  0.0445179 ,  0.0178041 ,  0.04014432]),
 'reward': array([ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , 

In [5]:
import os
import keras
from sklearn import preprocessing
for root, dirs, filenames in os.walk('/home/victor/ML/Smash/phillip-master/agents/FalconFalconBF/experience/'):
    for f in filenames:
        log = open(os.path.join(root, f),'rb')
        experience = pickle.load(log)
        #print(experience['state'])
        s, a = getpair(experience)
        x.append(s)
        y.append(a)
scaler = preprocessing.StandardScaler().fit(x)
x = scaler.transform(x)
y = keras.utils.to_categorical(y, 32)

Using TensorFlow backend.


In [6]:
len(y)

13006

In [7]:
#model.save('model')

In [8]:
model.evaluate(x_test, y_test)

NameError: name 'model' is not defined

In [9]:
from keras.models import load_model


In [10]:
m = load_model('model')

In [11]:
m.evaluate(x_test, y_test)

NameError: name 'x_test' is not defined

In [ ]:
t = flatten(experience['state'])
t = scaler.transform(t)
m.predict(np.expand_dims(t, axis=0))

In [ ]:
from sklearn.externals import joblib
scaler_filename = "scaler.save"
joblib.dump(scaler, scaler_filename) 

# And now to load...

scaler = joblib.load(scaler_filename) 

In [ ]:
len(flatten(experience['state']))

In [ ]:
experience['state']

In [ ]:
str(3)

In [9]:
import h5py
import numpy as np
from sklearn.externals import joblib
import keras
from sklearn import preprocessing

# And now| to load...

ry = []

In [10]:
def getData(scaler_filename = "scaler.save"):
    h5f = h5py.File('statedata.hdf5', 'r')
    x = []
    y = []
    for k in h5f.keys():
        x.append(h5f.get(k)[:-1])
        ry.append(int(h5f.get(k)[-1:]))
        y.append(int(h5f.get(k)[-1:]))
    scaler = preprocessing.StandardScaler().fit(x)
    x = scaler.transform(x)
    y = keras.utils.to_categorical(y, 32)
    
    joblib.dump(scaler, scaler_filename) 
    return x, y

In [11]:
scaler_filename = "scaler.save"
a, b = getData(scaler_filename)

In [13]:
s = a[0]
for i in range(9):
    s = np.append(s, a[i + 1])
x = []
y = []
for i in range(2000 - 20):
    x.append(s)
    y.append(b[i + 10])
    s = np.append(s[10:], a[i + 11])

In [6]:
x[0]

array([-1.14290222,  0.        ,  1.12215333, -0.0524464 ,  1.28233482,
        0.04564183, -0.82433745,  0.40004824,  0.        , -0.33373833,
       -0.21873933, -0.3765295 ,  0.21494066,  0.        ,  0.83233071,
        1.92081251,  0.02000362,  0.91062092, -0.02335591, -0.52766169,
        0.0584471 ,  0.        ,  0.        , -1.27736133,  0.        ,
       -0.95702727,  0.67824402,  1.12937036, -0.0581762 , -0.43128959,
       -0.80841894,  0.        , -0.16404891, -0.2160653 , -0.2578281 ,
        0.84274407, -0.14493656,  1.42930677, -0.01808614,  0.02193603,
       -0.91595711,  0.0183552 , -0.17605688,  0.39717317,  0.        ,
        0.        , -1.73132519,  0.        ,  0.        ,  0.        ,
        0.        , -0.58328903, -0.58328903,  1.04218225, -0.80398528,
        0.        ,  1.12215333,  0.4617831 , -0.45191542,  0.04564183,
       -0.66354271, -0.99379348,  0.        , -0.33373833, -0.21873933,
       -0.3765295 , -1.06027895,  0.        , -1.20144552,  1.20

In [ ]:
scaler = joblib.load(scaler_filename) 

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import regularizers
from keras.optimizers import SGD
model = Sequential([
    Dense(32, input_shape=(len(x[0]),), kernel_initializer='random_uniform'),
    Activation('relu'),
    Dropout(0.3),
    Dense(32, kernel_initializer='random_uniform'),
    Activation('softmax'),
])
model.compile(optimizer=keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
np.random.seed(70) 
x_train = x[len(x) // 5:]
y_train = y[len(y) // 5:]
x_test = x[:len(x) // 5]
y_test = y[:len(y) // 5]
x_train = np.array(x_train).astype('float32')
model.fit(x_train, y_train, epochs=200, batch_size=16, validation_data=(x_test, y_test))


Train on 10405 samples, validate on 2601 samples
Epoch 1/200
10405/10405 [==============================] - 1s - loss: 2.4061 - acc: 0.3679 - val_loss: 1.6302 - val_acc: 0.5352
Epoch 2/200
10405/10405 [==============================] - 1s - loss: 2.2686 - acc: 0.3831 - val_loss: 1.7200 - val_acc: 0.5040
Epoch 3/200
10405/10405 [==============================] - 1s - loss: 2.2050 - acc: 0.3841 - val_loss: 1.8150 - val_acc: 0.4679
Epoch 4/200
10405/10405 [==============================] - 1s - loss: 2.1635 - acc: 0.3862 - val_loss: 1.8373 - val_acc: 0.4521
Epoch 5/200
10405/10405 [==============================] - 1s - loss: 2.1343 - acc: 0.3816 - val_loss: 1.8964 - val_acc: 0.4429
Epoch 6/200
10405/10405 [==============================] - 1s - loss: 2.0954 - acc: 0.3970 - val_loss: 1.9011 - val_acc: 0.4364
Epoch 7/200
10405/10405 [==============================] - 1s - loss: 2.0932 - acc: 0.3962 - val_loss: 1.9772 - val_acc: 0.4260
Epoch 8/200
10405/10405 [==============================

10405/10405 [==============================] - 1s - loss: 1.6543 - acc: 0.4699 - val_loss: 3.4571 - val_acc: 0.2945
Epoch 65/200
10405/10405 [==============================] - 1s - loss: 1.6363 - acc: 0.4759 - val_loss: 3.5054 - val_acc: 0.2926
Epoch 66/200
10405/10405 [==============================] - 1s - loss: 1.6427 - acc: 0.4774 - val_loss: 3.5003 - val_acc: 0.3030
Epoch 67/200
10405/10405 [==============================] - 1s - loss: 1.6445 - acc: 0.4776 - val_loss: 3.5191 - val_acc: 0.2937
Epoch 68/200
10405/10405 [==============================] - 1s - loss: 1.6268 - acc: 0.4817 - val_loss: 3.5101 - val_acc: 0.3003
Epoch 69/200
10405/10405 [==============================] - 1s - loss: 1.6371 - acc: 0.4771 - val_loss: 3.5706 - val_acc: 0.2980
Epoch 70/200
10405/10405 [==============================] - 1s - loss: 1.6145 - acc: 0.4840 - val_loss: 3.6136 - val_acc: 0.2910
Epoch 71/200
10405/10405 [==============================] - 1s - loss: 1.6550 - acc: 0.4731 - val_loss: 3.5985

10405/10405 [==============================] - 3s - loss: 1.5321 - acc: 0.5055 - val_loss: 4.6386 - val_acc: 0.2641
Epoch 128/200
10405/10405 [==============================] - 3s - loss: 1.4865 - acc: 0.5230 - val_loss: 4.6325 - val_acc: 0.2668
Epoch 129/200
10405/10405 [==============================] - 3s - loss: 1.4942 - acc: 0.5231 - val_loss: 4.6673 - val_acc: 0.2691
Epoch 130/200
10405/10405 [==============================] - 2s - loss: 1.5185 - acc: 0.5123 - val_loss: 4.6976 - val_acc: 0.2614
Epoch 131/200
10405/10405 [==============================] - 3s - loss: 1.4913 - acc: 0.5147 - val_loss: 4.7477 - val_acc: 0.2645
Epoch 132/200
10405/10405 [==============================] - 2s - loss: 1.5255 - acc: 0.5070 - val_loss: 4.6933 - val_acc: 0.2684
Epoch 133/200
10405/10405 [==============================] - 2s - loss: 1.5032 - acc: 0.5176 - val_loss: 4.6980 - val_acc: 0.2661
Epoch 134/200
10405/10405 [==============================] - 2s - loss: 1.4792 - acc: 0.5146 - val_loss:

10405/10405 [==============================] - 2s - loss: 1.4377 - acc: 0.5352 - val_loss: 5.5688 - val_acc: 0.2572
Epoch 191/200
10405/10405 [==============================] - 2s - loss: 1.4428 - acc: 0.5346 - val_loss: 5.5504 - val_acc: 0.2572
Epoch 192/200
10405/10405 [==============================] - 2s - loss: 1.4252 - acc: 0.5349 - val_loss: 5.5581 - val_acc: 0.2553
Epoch 193/200
10405/10405 [==============================] - 2s - loss: 1.4092 - acc: 0.5416 - val_loss: 5.5234 - val_acc: 0.2564
Epoch 194/200
10405/10405 [==============================] - 1s - loss: 1.4205 - acc: 0.5363 - val_loss: 5.6359 - val_acc: 0.2561
Epoch 195/200
10405/10405 [==============================] - 2s - loss: 1.4301 - acc: 0.5388 - val_loss: 5.6162 - val_acc: 0.2545
Epoch 196/200
10405/10405 [==============================] - 2s - loss: 1.4393 - acc: 0.5396 - val_loss: 5.6319 - val_acc: 0.2630
Epoch 197/200
10405/10405 [==============================] - 2s - loss: 1.4182 - acc: 0.5370 - val_loss:

In [18]:
model.evaluate(x_test, y_test)

1792/2602 [===================>..........] - ETA: 0s

[9.0233256913990729, 0.23750960800530452]

In [ ]:
from keras.models import load_model
model = load_model('model')

In [ ]:
np.argmax(model.predict(np.expand_dims(x[0], axis=0)))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
gaussian_numbers = np.random.randn(1000)
plt.hist(ry, bins=32)
plt.title("Gaussian Histogram")
plt.xlabel("Value")
plt.ylabel("Frequency")

fig = plt.gcf()
fig.show()

In [ ]:
import tensorflow as tf
graph = tf.Graph()
graph.finalize()
tf_config = dict(
        allow_soft_placement=True,
        #log_device_placement=True,
      )
sess = tf.Session(
        graph=graph,
        config=tf.ConfigProto(**tf_config),
      ) 
#First let's load meta graph and restore weights
saver = tf.train.import_meta_graph('snapshot')

In [ ]:
variables = tf.global_variables()
saver = tf.train.Saver(variables)

In [ ]:
saver.restore(sess, 'snapshot')